# Proyecto: 'CDMX Un aire no tan bueno'
## Licenciatura en Ciencia de Datos
### Instituto de Investigaciones en Matemáticas Aplicadas y en Sistemas (IIMAS)
### Universidad Nacional Autónoma de México

Integrantes:

- Ortega Ibarra Jaime Jesús
- Martiñón Luna Jonathan José

----
# Pre-Procesamiento de datos.

## Índice de contenidos
* [Importando librerías](#E1)
* [Lectura de Datos](#E2)
* [Generando Funciones](#E3)
* [Pre-procesando Datos](#E4)
* [Discriminando datos](#E5)

----
<a name="E1"></a>
## Importando librerías necesarias

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import geopandas as gpd
from tabulate import tabulate
import re
import datetime
import swifter

----
<a name="E2"></a>
## Leyendo los datos

In [2]:
# Imeca2020
imeca_2020_df = pd.read_csv('../Datos/imeca_2020.csv')
#Calidad del aire
calidad_aire_df = pd.read_csv('../Datos/prueba_datos_calidad_aire.csv')
# Tiraderos clandestinos
tiraderos_clandestinos_df = pd.read_csv('../Datos/tiraderos-clandestinos-al-cierre-de-2017.csv')
# Nivel de plomo
nivel_plomo_df = pd.read_csv('../Datos/red_manual_plomo.csv',skiprows=8)
# Afluencia transporte
afluencia_transport_df = pd.read_csv('../Datos/afluencia-preliminar-en-transporte-publico.csv')
# Escuelas públicas
escuelas_publicas_df = pd.read_csv('../Datos/Escuelas/escuelas-publicas.csv')
# Escuelas privadas
escuelas_privadas_df = pd.read_csv('../Datos/Escuelas/escuelas-privadas.csv')
# Mercados
mercados_df = pd.read_csv('../Datos/mercados-publicos.csv')
# Parquímetros
parquimetros_df = pd.read_csv('../Datos/prueba-parquimetros.csv')
# Áreas Verdes
areas_verdes_df = pd.read_csv('../Datos/cdmx_areas_verdes_2017.csv')
# Áreas verdes con coordenadas
areas_verdes_Geo_df = gpd.read_file("../Datos/cdmx_areas_verdes_2017.geojson")
# Contador ciclistas
ciclistas_df = pd.read_csv('../Datos/contador-ciclistas.csv')
# Estaciones_Ecobici
estaciones_Ecobici_df = pd.read_csv('../Datos/estaciones-de-ecobici.csv')

#------------------
#Nota: Es importante descomprimir el archivo para poder leerlo.
# Contaminantes
contaminantes_df = pd.read_csv('../Datos/Contaminantes/contaminantes.csv')

----
<a name="E3"></a>
## Generando Funciones

In [3]:
def obten_dic_value_counts(objeto):
    '''
    @Author: Jonathan Martiñón
    
    @Descripción: 
                Función encargada de generar un diccionario
                a partir de los valores contados por pd.value_Counts()
                devuelve un diccionario donde el indice es el 'key' y
                el valor es 'value'
    '''
    Alcaldias = ['Benito Juarez','Tlahuac','Xochimilco','Tlalpan','Venustiano Carranza',
            'Milpa Alta', 'Miguel Hidalgo','Iztapalapa','Azcapotzalco','Cuajimalpa',
            'Cuauhtemoc', 'Alvaro Obregon', 'Magdalena Contreras', 'Coyoacan',
            'Gustavo A. Madero','Iztacalco']
    
    indices = objeto.index
    valores = objeto.values
    
    diccionario = dict(zip(indices,valores))
    
    for alcaldia in Alcaldias:
        if alcaldia not in indices:
            diccionario.update({alcaldia:0})
    
    return diccionario

In [4]:
def obten_alcaldia(direccion):
    '''
    @Author: Jonathan Martiñón
    
    @Descripción
            Para el dataset de escuelas tenemos cierta estructura
            donde la delegación viene incluída en el Domicilio.
            "DELEGACION x,". La presente función se encarga
            de extraer esa delegación.
    '''
    # Si tiene DELEGACION, hacemos un split
    dire = re.split("DELEGACION ",direccion)
    
    # Si el split se logra, tendrá al menos 2 elementos
    if len(dire)>1:
        # Nos interesa la segunda parte, que lleva una coma seguida
        # Lo quitamos y ajustamos a formato title
        alcaldia=dire[1].split(",")[0].title()
        
        #Datos atípicos
        if alcaldia == 'Cuajimalpa De Morelos':
            alcaldia = 'Cuajimalpa'
            
        elif alcaldia == 'Azcapotzalco Obregon':
            alcaldia = 'Azcapotzalco'
        elif alcaldia == 'Alvaro Obregón':
            alcaldia = 'Alvaro Obregon'
            
    # En caso de no encontrar delegación
    else:
        alcaldia = "No aplica"
    return alcaldia

In [5]:
def elimina_acentos_alcaldia(alcaldia):
    '''
    @Autor: Jonathan Martiñón
    
    @Descripción: 
            Función encargada de eliminar acentos en vocales para 
            una cadena.
    @Ejemplo
    alcaldia = 'Benito Juárez'
    elimina_acentos_alcaldia(alcaldia)
    >>> 'Benito Juarez'
    '''
    
    tildes = {"á":"a","é":"e","í":"i","ó":"o"}
    alcaldia = alcaldia.lower()
    for i in range(len(alcaldia)):
        
        if alcaldia[i] in list(tildes.keys()):
            
            alcaldia = alcaldia.replace(alcaldia[i],tildes[alcaldia[i]])
            
    return alcaldia.title()

In [6]:
def estacion_a_alcaldia(estacion):
    est_a_alc = {"Santa fe":"Cuajimalpa","Mguel Hidalgo":"Miguel Hidalgo",
                 "Pedregal":"Alvaro Obregon","Ajusco Medio":"Miguel Hidalgo",
                 "Coyoacán":"Coyoacan","Centro de Ciencias de la Atmósfera":"Coyoacan",
                 "UAM Xochimilco":"Xochimilco","UAM Iztapalapa":"Iztapalapa",
                 "Benito Juárez":"Benito Juarez","Santiago Acahualtepec":"Iztapalapa",
                 "Camarones":"Azcapotzalco","Hospital General de México":"Cuauhtemoc",
                 "Merced":"Venustiano Carranza","Ajusco":"Tlalpan"}
    
    Na = ['Nezahualcóyotl', 'San Agustín', 'Villa de las Flores','Acolman', 'Montecillo',
         'Investigaciones Nucleares','FES Aragón', 'Atizapan', 'Chalco','Tultitlán',
          'La Presa','San Juan Aragón','Xalostoc','Tlalnepantla','FES Acatlán',
         'Cuautitlán','Los Laureles']
    if estacion in Na:
        return "No aplica"
    elif estacion in list(est_a_alc.keys()):
        return est_a_alc[estacion]
    else:
        return estacion
        

----
<a name="E4"></a>
## Pre-procesamiento

In [7]:
print(list(contaminantes_df.columns))
print(contaminantes_df.shape)
contaminantes_df.head()

['Fecha', 'Nombre de estación', 'Clave de estación', 'Clave Parámetro', 'Nombre de parámetro', 'Unidades parámetros', 'Valor', 'ID Unidad', 'Clave de unidad', 'Nombre de unidad', 'Altura Estación', 'Observación Estación', 'ID Estación', 'Latitud', 'Longitud', 'Coordenadas']
(469560, 16)


,Fecha,Nombre de estación,Clave de estación,Clave Parámetro,Nombre de parámetro,Unidades parámetros,Valor,ID Unidad,Clave de unidad,Nombre de unidad,Altura Estación,Observación Estación,ID Estación,Latitud,Longitud,Coordenadas
0,2020-02-15T20:00:00-06:00,Santa fe,SFE,NO2,Dióxido de nitrógeno,1,18.0,1,ppb,partes por billón,2599,NaN,484090040309,19.357357,-99.262865,"19.357357,-99.262865"
1,2020-02-15T20:00:00-06:00,Nezahualcóyotl,NEZ,NO2,Dióxido de nitrógeno,1,27.0,1,ppb,partes por billón,2235,NaN,484150580115,19.393734,-99.028212,"19.393734,-99.028212"
2,2020-02-15T20:00:00-06:00,San Agustín,SAG,O3,Ozono,1,8.0,1,ppb,partes por billón,2241,NaN,484150330327,19.532968,-99.030324,"19.532968,-99.030324"
3,2020-02-15T20:00:00-06:00,Villa de las Flores,VIF,O3,Ozono,1,6.0,1,ppb,partes por billón,2242,NaN,484150200109,19.658223,-99.096590,"19.658223,-99.09659"
4,2020-02-15T20:00:00-06:00,Milpa Alta,MPA,O3,Ozono,1,24.0,1,ppb,partes por billón,2594,NaN,484090090104,19.176900,-98.990189,"19.1769,-98.990189"


In [8]:
Alcaldias = ['Benito Juarez','Tlahuac','Xochimilco','Tlalpan','Venustiano Carranza',
            'Milpa Alta', 'Miguel Hidalgo','Iztapalapa','Azcapotzalco','Cuajimalpa',
            'Cuauhtemoc', 'Alvaro Obregon', 'Magdalena Contreras', 'Coyoacan',
            'Gustavo A. Madero','Iztacalco']

In [9]:
#Obtenemos las delegaciones que poseen una estación
Estac_list = list(contaminantes_df['Nombre de estación'].apply(estacion_a_alcaldia).unique())

In [10]:
#Convertimos los valores obtenidos a un diccionario
Estac_alc_dic = {alcaldia:(1 if alcaldia in Estac_list else 0) for alcaldia in Alcaldias}

In [11]:
# Obtenemos la cantidad de áreas verdes por delegación
areas_verdes_dic = obten_dic_value_counts(areas_verdes_df['delegacion'].apply(elimina_acentos_alcaldia).replace('Cuajimalpa De Morelos','Cuajimalpa').value_counts())

In [12]:
#Obtenemos la cantidad de parquímetros por alcaldía
parquimetros_dic = obten_dic_value_counts(parquimetros_df['Alcaldía'].apply(elimina_acentos_alcaldia).value_counts())

In [13]:
#Obtenemos la cantidad de mercados por alcaldía
mercados_dic = obten_dic_value_counts(mercados_df['alcaldia'].apply(str.title).value_counts())

In [14]:
# Obtenemos la cantidad de escuelas privadas por alcaldia
escuelas_privadas_dic = obten_dic_value_counts(escuelas_privadas_df['ALCALDÍA'].apply(str.title).replace('Cuajimalpa De Morelos',
                                                          'Cuajimalpa').replace('La Magdalena Contreras',
                                                                                'Magdalena Contreras').value_counts())

In [15]:
#Obtenemos la Alcaldía a partir del domicilio
escuelas_publicas_df['Alcaldia'] = escuelas_publicas_df['Domicilio con nombre'].apply(obten_alcaldia)

#Eliminamos aquellos lugares que no corresponden a la CDMX
escuelas_publicas_df.drop(escuelas_publicas_df[escuelas_publicas_df['Alcaldia']=='No aplica'].index
                          ,axis=0, inplace=True)

In [16]:
escuelas_publicas_dic = obten_dic_value_counts(escuelas_publicas_df['Alcaldia'].value_counts())

In [17]:
tiraderos_dic = obten_dic_value_counts(tiraderos_clandestinos_df.groupby('alcaldia')['alcaldia'].count())

----
<a name="E5"></a>
## Discriminando Datos

De los valores que estamos seguros que nos servirán será la alcaldía

In [18]:
#Creamos un dataframe para extraer las características propias a cada alcaldía
data_Alcaldias_df = pd.DataFrame(columns=['Alcaldia','Tiraderos','Escuelas_publicas',
                                'Escuelas_privadas','Mercados','Parquimetros',
                                'Areas_verdes','Estacion_climatologica'])

In [19]:
#Llenamos nuestro dataframe con sus correspondientes datos
for alc in Alcaldias:
    data_Alcaldias_df.loc[len(data_Alcaldias_df)] = [alc,tiraderos_dic[alc],escuelas_publicas_dic[alc],
                                escuelas_privadas_dic[alc],mercados_dic[alc],
                                parquimetros_dic[alc],areas_verdes_dic[alc],
                                Estac_alc_dic[alc]]

#Convertimos el nombre de la alcaldía a nuestro índice    
data_Alcaldias_df.set_index('Alcaldia',inplace=True)

In [20]:
#Una pequeña visualización de la forma en que van quedando los datos
data_Alcaldias_df.head(2)

,Tiraderos,Escuelas_publicas,Escuelas_privadas,Mercados,Parquimetros,Areas_verdes,Estacion_climatologica
Alcaldia,,,,,,,
Benito Juarez,147,68,294,16,640,484,1
Tlahuac,35,120,103,18,0,581,1


In [21]:
def obten_valor_col(alcaldia,datos,col):
    '''
    @Autor: Jonathan Martiñón
    
    @Parametros:
    
    alcaldia  índice del DataFrame con valores de la alcaldía   STR
    datos     DataFrame con alcaldías como índices              pd.DataFrame
    col       Columna del DataFrame a extraer valor
    
    @Descripción:
            Función encargada de extraer características de la 
            alcaldía.
    @Devuelve:
    
    value   Valor obtenido del índice 'alcaldía' y la columna 'col'
    '''
    # Obtenemos el valor con el indice de la alcaldia actual
    # y columna 'col'
    value = datos[col][alcaldia]
    
    return value
    

In [22]:
def Genera_dataset(data,data_al):
    '''
    @Author: Jonathan Martiñón
    
    @Parametros:
    
    data1    Dataset que contiene los contaminantes              pd.DataFrame
    data2    Dataset que contiene informacion de las alcaldias   pd.DataFrame
    
    @Descripción:
            Función encargada de devolver el dataset Final.
            
    @Devuelve:
    
    data_df  DataFrame con los datos Finales.   LIST
    '''
    
    #Copiamos el dataset
    data_df = data.copy()
    
    #Obtenemos las alcaldías a partir de la estación
    data_df['Alcaldia'] = data_df['Nombre de estación'].swifter.apply(estacion_a_alcaldia)
    
    #Eliminamos aquellas alcaldías que no aplican a nuestro análisis
    data_df.drop(data_df[data_df['Alcaldia']=='No aplica'].index,axis=0,inplace=True)
    
    #Restauramos nuestro índice
    data_df.reset_index(drop = True, inplace=True) 
    
    #Convertimos la fecha a formato datetime para trabajar con ella
    data_df['Fecha']= pd.to_datetime(data_df['Fecha'])
    
    #Extraemos Día, Mes, Año y Hora de la Fecha
    data_df['Dia'] = data_df['Fecha'].swifter.apply(lambda x: x.day)
    data_df['Mes'] = data_df['Fecha'].swifter.apply(lambda x: x.month)
    data_df['Año'] = data_df['Fecha'].swifter.apply(lambda x: x.year)
    data_df['Hora'] = data_df['Fecha'].swifter.apply(lambda x: x.hour)
    
    #Eliminamos la fecha, pues ya no nos sirve
    data_df.drop(['Fecha'],axis=1,inplace=True)
    
    #Extraemos la cantidad de tiraderos por alcaldía
    data_df['Tiraderos'] = data_df['Alcaldia'].swifter.apply(obten_valor_col,
                                                             datos=data_al,
                                                             col='Tiraderos').values
    #Extraemos la cantidad de escuelas públicas por alcaldía
    data_df['Escuelas_publicas'] = data_df['Alcaldia'].swifter.apply(obten_valor_col,
                                                                     datos=data_al,
                                                                     col='Escuelas_publicas').values
    #Extraemos la cantidad de escuelas privadas por alcaldía
    data_df['Escuelas_privadas'] = data_df['Alcaldia'].swifter.apply(obten_valor_col,
                                                                     datos=data_al,
                                                                     col='Escuelas_privadas').values
    #Extraemos la cantidad de mercados por alcaldía
    data_df['Mercados'] = data_df['Alcaldia'].swifter.apply(obten_valor_col,
                                                            datos=data_al,
                                                            col='Mercados').values
    #Extraemos la cantidad de parquímetros por alcaldía
    data_df['Parquimetros'] = data_df['Alcaldia'].swifter.apply(obten_valor_col,
                                                                datos=data_al,
                                                                col='Parquimetros').values
    #Extraemos la cantidad de áreas verdes por alcaldía
    data_df['Areas_verdes'] = data_df['Alcaldia'].swifter.apply(obten_valor_col,
                                                                datos=data_al,
                                                                col='Areas_verdes').values
    #Extraemos la cantidad de estaciones climatológicas por alcaldía
    data_df['Estacion_climatologica'] = data_df['Alcaldia'].apply(obten_valor_col,
                                                                  datos=data_al,
                                                                  col='Estacion_climatologica').values
    # Devolvemos el dataset generado
    return data_df

In [23]:
# Obtenemos los datos importantes del dataset
# de contaminantes
data_df = contaminantes_df[['Fecha','Nombre de estación','Clave de estación','Clave Parámetro',
                   'Nombre de parámetro','Unidades parámetros','Valor','ID Unidad',
                            'Clave de unidad']].copy(deep=True) 

In [24]:
# Damos una visualización de los datos
print(f"Contamos con {data_df.shape[0]} registros y {data_df.shape[1]} columnas")
data_df.head(2)

Contamos con 469560 registros y 9 columnas


,Fecha,Nombre de estación,Clave de estación,Clave Parámetro,Nombre de parámetro,Unidades parámetros,Valor,ID Unidad,Clave de unidad
0,2020-02-15T20:00:00-06:00,Santa fe,SFE,NO2,Dióxido de nitrógeno,1,18.0,1,ppb
1,2020-02-15T20:00:00-06:00,Nezahualcóyotl,NEZ,NO2,Dióxido de nitrógeno,1,27.0,1,ppb


In [25]:
#Generamos nuestro nuevo dataset con valores personalizados
# por alcaldía
data_df = Genera_dataset(data_df,data_Alcaldias_df)

In [26]:
#Almacenamos nuestro dataset
# data_df.to_csv('./new_data.csv')

In [27]:
# Damos una visualización de los nuevos datos
print(f"Contamos con {data_df.shape[0]} registros y {data_df.shape[1]} columnas")
data_df.head()

Contamos con 248976 registros y 20 columnas


,Nombre de estación,Clave de estación,Clave Parámetro,Nombre de parámetro,Unidades parámetros,Valor,ID Unidad,Clave de unidad,Alcaldia,Dia,Mes,Año,Hora,Tiraderos,Escuelas_publicas,Escuelas_privadas,Mercados,Parquimetros,Areas_verdes,Estacion_climatologica
0,Santa fe,SFE,NO2,Dióxido de nitrógeno,1,18.0,1,ppb,Cuajimalpa,15,2,2020,20,5,35,172,5,0,448,1
1,Milpa Alta,MPA,O3,Ozono,1,24.0,1,ppb,Milpa Alta,15,2,2020,20,13,28,32,10,0,130,1
2,Mguel Hidalgo,MGH,O3,Ozono,1,27.0,1,ppb,Miguel Hidalgo,15,2,2020,20,69,137,223,19,1382,925,1
3,Pedregal,PED,O3,Ozono,1,19.0,1,ppb,Alvaro Obregon,15,2,2020,20,77,172,321,16,83,721,1
4,Ajusco Medio,AJM,O3,Ozono,1,NaN,1,ppb,Miguel Hidalgo,15,2,2020,20,69,137,223,19,1382,925,1
